## Calculating Codon Adaptation Index according (CAI) according to Sharp et al. 1987

The codon adaptation index (CAI) measures the level of adaptation of a gene to a certain organism. 

It was first introduced by [Sharp 1987](https://www.ncbi.nlm.nih.gov/pubmed/3547335). CAI measures the deviation of the codons in a given protein coding gene sequence with respect to the codons used in a reference set of genes. See the [wikipedia](https://en.wikipedia.org/wiki/Codon_Adaptation_Index) article on CAI.

There are at least two python implementations for calculating CAI.

[Biopython](http://biopython.org) has a module called [CodonUsage](http://biopython.org/DIST/docs/api/Bio.SeqUtils.CodonUsage.CodonAdaptationIndex-class.html). This module does not contain any of the necessary reference data in order to calculate CAI for yeast. 
Worse, the fidelity of the Biopython module had been questioned in this exchange on [Biostars](https://www.biostars.org/p/290485/).

The python module [CAI](https://pypi.org/project/CAI/) is an alternative implementation.

The purpose of this notebook is to test the fidelity of both the Biopython and CAI modules using the original data from Sharp 1987.

Codon adaptation data was tabulated in Table 1 [Sharp 1987](https://www.ncbi.nlm.nih.gov/pubmed/3547335).

![Sharp Table 1](sharp_table_1.png)

The Sharp Table 1 was copied and edited from the pdf version of the publication. 

The order of the columns remains the same.

RSCU and weights (w) are tabulated for 61 of the 64 codons (missing are the stop codons TAA, TAG and TGA).






In [1]:
sharp_table_1 = """
AA Tri RSCUe we RSCUy wy
Phe TTT 0.456 0.296 0.203 0.113
Phe TTC 1.544 1.000 1.797 1.000
Leu TTA 0.106 0.020 0.601 0.117
Leu TTG 0.106 0.020 5.141 1.000
Leu CTT 0.225 0.042 0.029 0.006
Leu CTC 0.198 0.037 0.014 0.003
Leu CTA 0.040 0.007 0.200 0.039
Leu CTG 5.326 1.000 0.014 0.003
Ile ATT 0.466 0.185 1.352 0.823
Ile ATC 2.525 1.000 1.643 1.000
Ile ATA 0.008 0.003 0.005 0.003
Met ATG 1.000 1.000 1.000 1.000
Val GTT 2.244 1.000 2.161 1.000
Val GTC 0.148 0.066 1.796 0.831
Val GTA 1.111 0.495 0.004 0.002
Val GTG 0.496 0.221 0.039 0.018
Ser TCT 2.571 1.000 3.359 1.000
Ser TCC 1.912 0.744 2.327 0.693
Ser TCA 0.198 0.077 0.122 0.036
Ser TCG 0.044 0.017 0.017 0.005
Pro CCT 0.231 0.070 0.179 0.047
Pro CCC 0.038 0.012 0.036 0.009
Pro CCA 0.442 0.135 3.776 1.000
Pro CCG 3.288 1.000 0.009 0.002
Thr ACT 1.804 0.965 1.899 0.921
Thr ACC 1.870 1.000 2.063 1.000
Thr ACA 0.141 0.076 0.025 0.012
Thr ACG 0.185 0.099 0.013 0.006
Ala GCT 1.877 1.000 3.005 1.000
Ala GCC 0.228 0.122 0.948 0.316
Ala GCA 1.099 0.586 0.044 0.015
Ala GCG 0.796 0.424 0.004 0.001
Tyr TAT 0.386 0.239 0.132 0.071
Tyr TAC 1.614 1.000 1.868 1.000
His CAT 0.451 0.291 0.394 0.245
His CAC 1.549 1.000 1.606 1.000
Gln CAA 0.220 0.124 1.987 1.000
Gln CAG 1.780 1.000 0.013 0.007
Asn AAT 0.097 0.051 0.100 0.053
Asn AAC 1.903 1.000 1.900 1.000
Lys AAA 1.596 1.000 0.237 0.135
Lys AAG 0.404 0.253 1.763 1.000
Asp GAT 0.605 0.434 0.713 0.554
Asp GAC 1.395 1.000 1.287 1.000
Glu GAA 1.589 1.000 1.968 1.000
Glu GAG 0.411 0.259 0.032 0.016
Cys TGT 0.667 0.500 1.857 1.000
Cys TGC 1.333 1.000 0.143 0.077
Trp TGG 1.000 1.000 1.000 1.000
Arg CGT 4.380 1.000 0.718 0.137
Arg CGC 1.561 0.356 0.008 0.002
Arg CGA 0.017 0.004 0.008 0.002
Arg CGG 0.017 0.004 0.008 0.002
Ser AGT 0.220 0.085 0.070 0.021
Ser AGC 1.055 0.410 0.105 0.031
Arg AGA 0.017 0.004 5.241 1.000
Arg AGG 0.008 0.002 0.017 0.003
Gly GGT 2.283 1.000 3.898 1.000
Gly GGC 1.652 0.724 0.077 0.020
Gly GGA 0.022 0.010 0.009 0.002
Gly GGG 0.043 0.019 0.017 0.004"""

Pandas can turn this text table into a dataframe

In [2]:
import pandas as pd
from io import StringIO

In [3]:
sharp_df = pd.read_csv(StringIO(sharp_table_1.strip()), sep=" ")

The resulting dataframe seems to reflect the original table above.

In [4]:
sharp_df

,AA,Tri,RSCUe,we,RSCUy,wy
0,Phe,TTT,0.456,0.296,0.203,0.113
1,Phe,TTC,1.544,1.000,1.797,1.000
2,Leu,TTA,0.106,0.020,0.601,0.117
3,Leu,TTG,0.106,0.020,5.141,1.000
4,Leu,CTT,0.225,0.042,0.029,0.006
...,...,...,...,...,...,...
56,Arg,AGG,0.008,0.002,0.017,0.003
57,Gly,GGT,2.283,1.000,3.898,1.000
58,Gly,GGC,1.652,0.724,0.077,0.020
59,Gly,GGA,0.022,0.010,0.009,0.002


Each row is combined into a dict

In [5]:
RSCUe = dict(zip(sharp_df["Tri"],round(sharp_df["RSCUe"],3)))
we = dict(zip(sharp_df["Tri"],round(sharp_df["we"],3)))
RSCUy = dict(zip(sharp_df["Tri"],round(sharp_df["RSCUy"],3)))
wy = dict(zip(sharp_df["Tri"],round(sharp_df["wy"],3)))

In [6]:
RSCUe["TTT"], we["TTT"], RSCUy["TTT"], wy["TTT"]

(0.456, 0.296, 0.203, 0.113)

The `we` dict should be the same as the `SharpEcoliIndex` from biopython

In [7]:
from Bio.SeqUtils.CodonUsageIndices import SharpEcoliIndex

This is actually so:

In [8]:
SharpEcoliIndex == we

True

The CAI is calculated for the _E. coli_ rpsU gene given as an example on page 1285 in Sharp 1987. 
This sequence is available from Genbank (NC_000913 REGION: 3210781..3210996).

    atg ccg gta att aaa gta cgtgaaaacgagccgttcgacgtagctctgcgtcgctt
        ||| ||| ||| ||| |||
       .CCG.GTA.ATT.AAA.GTA.

    caagcgttcctgcgaaaaagcaggtgttctggcggaagttcgtcgtcgtgagttctatgaaaaaccgactaccgaacgtaagcg
    cgctaaagcttctgcagtgaaacgtcacgcgaagaaactggctcgcgaaaacgcacgccgcactcgtctgtactaa

In [9]:
from pydna.genbank import genbank

In [10]:
rpsU = str(genbank("NC_000913 REGION: 3210781..3210996").seq)
print(rpsU)

ATGCCGGTAATTAAAGTACGTGAAAACGAGCCGTTCGACGTAGCTCTGCGTCGCTTCAAGCGTTCCTGCGAAAAAGCAGGTGTTCTGGCGGAAGTTCGTCGTCGTGAGTTCTATGAAAAACCGACTACCGAACGTAAGCGCGCTAAAGCTTCTGCAGTGAAACGTCACGCGAAGAAACTGGCTCGCGAAAACGCACGCCGCACTCGTCTGTACTAA


The data can be used to test the prediction of the biopython module.

In [11]:
from Bio.SeqUtils.CodonUsage import CodonAdaptationIndex

In [12]:
cai = CodonAdaptationIndex()

In [13]:
cai.set_cai_index(we)

In [14]:
round(cai.cai_for_gene(rpsU), 3)

0.723

The biopython module show a small difference, 0.723 instead of 0.726

In [15]:
from CAI import CAI

In [16]:
round(CAI(rpsU, weights=we),3)

0.726

The CAI module calculates exactly the same value for rpsU as the one given in the paper.

Some other examples of CAI values were calculated for some genes from _E. coli_ and _S. cerevisiae_.

These were listed in table 2 of Sharp 1987.

[![Sharp Table 2](sharp_table_2.png)](sharp_table_2.png)

The table was transferred to text format:

| E.coli | E.coli      | yeast      | yeast       |
|--------|-------------|------------|-------------|
| gene   | CAI         | gene       | CAI         |
| 17 RPs | 0.467-0.813 | 16 RPs     | 0.529-0.915 |
| rpsU   | 0.726       | histones   | 0.529-0.915 |
| rpoD   | 0.582       | 2u plasmid | 0.099-0.106 |
| dnaG   | 0.271       | *GAL4*     | 0.116       |
| lacI   | 0.296       | *PPR1*     | 0.114       |
| trpR   | 0.267       | *GPD1*     | 0.929       |
| lpp    | 0.849       | matA2      | 0.098       |
| hsdS   | 0.218       |            |             |


The yeast genes GAL4, PPR1 and GPD1 genes are easy to identify:

- [GAL4](https://www.yeastgenome.org/locus/GAL4)
- [PPR1](https://www.yeastgenome.org/locus/PPR1)
- [TDH3](https://www.yeastgenome.org/locus/TDH3)

Seguences for these are available from pygenome.

In [17]:
from pygenome import saccharomyces_cerevisiae as sg

In [18]:
GAL4 = str(sg.stdgenes["GAL4"].cds().seq)
PPR1 = str(sg.stdgenes["PPR1"].cds().seq)
GPD1 = str(sg.stdgenes["TDH3"].cds().seq)

AttributeError: module 'pygenome.saccharomyces_cerevisiae' has no attribute 'stdgenes'

The biopython cai index is changed for the wy dict:

In [19]:
cai.set_cai_index(wy)

In [20]:
print(round(cai.cai_for_gene(GAL4),3))
print(round(cai.cai_for_gene(PPR1),3))
print(round(cai.cai_for_gene(GPD1),3))

0.116
0.114
0.924


The same genes and the same data produce almost the same result using the CAI module

In [21]:
print(round(CAI(GAL4, weights=wy),3))
print(round(CAI(PPR1, weights=wy),3)) 
print(round(CAI(GPD1, weights=wy),3)) 

0.116
0.115
0.924


The cai mudule can alternatively accept RSCUs, which produce the same result.

In [22]:
print(round(CAI(GAL4, RSCUs=RSCUy),3))
print(round(CAI(PPR1, RSCUs=RSCUy),3)) 
print(round(CAI(GPD1, RSCUs=RSCUy),3)) 

0.116
0.115
0.924


There was a small differences for PPR1 (<1% for the CAI module) and a lower value for GDP/TDH3 (0.5%) between the data given in the paper and the result of the CAI and biopython modules.

| Gene  | Sharp | CAI(wy)| Biopython(wy)|
| ----- | ----- | -----  | ------------ |
| GAL4  | 0.116 | 0.116  | 0.116        |
| PPR1  | 0.114 | 0.115  | 0.115        |
| GPD1  | 0.929 | 0.924  | 0.924        |








Some companies such as [Genscript](https://www.genscript.com)
offer services to optimize a gene for a specific host (for example [here](https://www.genscript.com/tools/rare-codon-analysis)). 

A part of this analysis usually involves calculating CAI.to see if a gene needs to be codon optimized for expression.
The Genscript website implies that a CAI of 0.8-1.0 is desirable.

| gene  | CAI(wy) | Genscript |
|-------| ------- |-----------|
| GAL4  |  0.116  | 0.71      |
| PPR1  |  0.115  | 0.73      |
| GPD1  |  0.924  | 0.82      |

The Genscript website gave very different values (2021-03-17) for the genes, the lower values are much higher.
The most probable cause for this is the use of a different reference dataset.
This tool does not reveal which reference data was used.

A popular resource for codon usage data is the "Codon Usage Database" hosted by the Kazusa DNA Research Institute. 
CAI reference data for _S. cerevisiae_ can be found [here](https://www.kazusa.or.jp/codon/cgi-bin/showcodon.cgi?species=4932).

The CAI reference data in "Kazusa" format:

    Saccharomyces cerevisiae [gbpln]: 14411 CDS's (6534504 codons)
    fields: [triplet] [amino acid] [fraction] [frequency: per thousand] ([number])
    
    UUU F 0.59 26.1 (170666)  UCU S 0.26 23.5 (153557)  UAU Y 0.56 18.8 (122728)  UGU C 0.63  8.1 ( 52903)
    UUC F 0.41 18.4 (120510)  UCC S 0.16 14.2 ( 92923)  UAC Y 0.44 14.8 ( 96596)  UGC C 0.37  4.8 ( 31095)
    UUA L 0.28 26.2 (170884)  UCA S 0.21 18.7 (122028)  UAA * 0.47  1.1 (  6913)  UGA * 0.30  0.7 (  4447)
    UUG L 0.29 27.2 (177573)  UCG S 0.10  8.6 ( 55951)  UAG * 0.23  0.5 (  3312)  UGG W 1.00 10.4 ( 67789)

    CUU L 0.13 12.3 ( 80076)  CCU P 0.31 13.5 ( 88263)  CAU H 0.64 13.6 ( 89007)  CGU R 0.14  6.4 ( 41791)
    CUC L 0.06  5.4 ( 35545)  CCC P 0.15  6.8 ( 44309)  CAC H 0.36  7.8 ( 50785)  CGC R 0.06  2.6 ( 16993)
    CUA L 0.14 13.4 ( 87619)  CCA P 0.42 18.3 (119641)  CAA Q 0.69 27.3 (178251)  CGA R 0.07  3.0 ( 19562)
    CUG L 0.11 10.5 ( 68494)  CCG P 0.12  5.3 ( 34597)  CAG Q 0.31 12.1 ( 79121)  CGG R 0.04  1.7 ( 11351)

    AUU I 0.46 30.1 (196893)  ACU T 0.35 20.3 (132522)  AAU N 0.59 35.7 (233124)  AGU S 0.16 14.2 ( 92466)
    AUC I 0.26 17.2 (112176)  ACC T 0.22 12.7 ( 83207)  AAC N 0.41 24.8 (162199)  AGC S 0.11  9.8 ( 63726)
    AUA I 0.27 17.8 (116254)  ACA T 0.30 17.8 (116084)  AAA K 0.58 41.9 (273618)  AGA R 0.48 21.3 (139081)
    AUG M 1.00 20.9 (136805)  ACG T 0.14  8.0 ( 52045)  AAG K 0.42 30.8 (201361)  AGG R 0.21  9.2 ( 60289)

    GUU V 0.39 22.1 (144243)  GCU A 0.38 21.2 (138358)  GAU D 0.65 37.6 (245641)  GGU G 0.47 23.9 (156109)
    GUC V 0.21 11.8 ( 76947)  GCC A 0.22 12.6 ( 82357)  GAC D 0.35 20.2 (132048)  GGC G 0.19  9.8 ( 63903)
    GUA V 0.21 11.8 ( 76927)  GCA A 0.29 16.2 (105910)  GAA E 0.70 45.6 (297944)  GGA G 0.22 10.9 ( 71216)
    GUG V 0.19 10.8 ( 70337)  GCG A 0.11  6.2 ( 40358)  GAG E 0.30 19.2 (125717)  GGG G 0.12  6.0 ( 39359)

The setuptools package [python-codon-tables](https://pypi.org/project/python-codon-tables/) 
claims that it can access this data directly:

In [19]:
# uncomment below to run pip in jupyter notebook
#     %pip install python-codon-tables

In [20]:
import python_codon_tables as pct

ModuleNotFoundError: No module named 'python_codon_tables'

In [25]:
table = pct.get_codons_table('s_cerevisiae_4932')

In [26]:
table

{'*': {'TAA': 0.47, 'TAG': 0.23, 'TGA': 0.3},
 'A': {'GCA': 0.29, 'GCC': 0.22, 'GCG': 0.11, 'GCT': 0.38},
 'C': {'TGC': 0.37, 'TGT': 0.63},
 'D': {'GAC': 0.35, 'GAT': 0.65},
 'E': {'GAA': 0.7, 'GAG': 0.3},
 'F': {'TTC': 0.41, 'TTT': 0.59},
 'G': {'GGA': 0.22, 'GGC': 0.19, 'GGG': 0.12, 'GGT': 0.47},
 'H': {'CAC': 0.36, 'CAT': 0.64},
 'I': {'ATA': 0.27, 'ATC': 0.26, 'ATT': 0.46},
 'K': {'AAA': 0.58, 'AAG': 0.42},
 'L': {'CTA': 0.14,
  'CTC': 0.06,
  'CTG': 0.11,
  'CTT': 0.13,
  'TTA': 0.28,
  'TTG': 0.29},
 'M': {'ATG': 1.0},
 'N': {'AAC': 0.41, 'AAT': 0.59},
 'P': {'CCA': 0.42, 'CCC': 0.15, 'CCG': 0.12, 'CCT': 0.31},
 'Q': {'CAA': 0.69, 'CAG': 0.31},
 'R': {'AGA': 0.48,
  'AGG': 0.21,
  'CGA': 0.07,
  'CGC': 0.06,
  'CGG': 0.04,
  'CGT': 0.14},
 'S': {'AGC': 0.11,
  'AGT': 0.16,
  'TCA': 0.21,
  'TCC': 0.16,
  'TCG': 0.1,
  'TCT': 0.26},
 'T': {'ACA': 0.3, 'ACC': 0.22, 'ACG': 0.14, 'ACT': 0.35},
 'V': {'GTA': 0.21, 'GTC': 0.21, 'GTG': 0.19, 'GTT': 0.39},
 'W': {'TGG': 1.0},
 'Y': {'TAC

This format does not seem to be directly compatible with CAI or biopython modules.

It needs to be a dict with the string triplet as key.

Further, the values seem to be the fractions in the kazusa table above.

We can fix this forming a dict with the triplets as keys and values that are fractions scaled to 1 for the largest fraction.

In [27]:
codon_tables_w = {}
for dct in table.values():
    m = max(dct.values())
    for trip, w in dct.items():
        codon_tables_w[trip] = w/m

The resulting dict weights has the correct format for the CAI and biopython modules. 

In [28]:
print(codon_tables_w)

{'TAA': 1.0, 'TAG': 0.4893617021276596, 'TGA': 0.6382978723404256, 'GCA': 0.763157894736842, 'GCC': 0.5789473684210527, 'GCG': 0.2894736842105263, 'GCT': 1.0, 'TGC': 0.5873015873015873, 'TGT': 1.0, 'GAC': 0.5384615384615384, 'GAT': 1.0, 'GAA': 1.0, 'GAG': 0.4285714285714286, 'TTC': 0.6949152542372882, 'TTT': 1.0, 'GGA': 0.46808510638297873, 'GGC': 0.4042553191489362, 'GGG': 0.2553191489361702, 'GGT': 1.0, 'CAC': 0.5625, 'CAT': 1.0, 'ATA': 0.5869565217391305, 'ATC': 0.5652173913043478, 'ATT': 1.0, 'AAA': 1.0, 'AAG': 0.7241379310344828, 'CTA': 0.48275862068965525, 'CTC': 0.20689655172413793, 'CTG': 0.37931034482758624, 'CTT': 0.4482758620689656, 'TTA': 0.9655172413793105, 'TTG': 1.0, 'ATG': 1.0, 'AAC': 0.6949152542372882, 'AAT': 1.0, 'CCA': 1.0, 'CCC': 0.35714285714285715, 'CCG': 0.2857142857142857, 'CCT': 0.7380952380952381, 'CAA': 1.0, 'CAG': 0.44927536231884063, 'AGA': 1.0, 'AGG': 0.4375, 'CGA': 0.14583333333333334, 'CGC': 0.125, 'CGG': 0.08333333333333334, 'CGT': 0.2916666666666667, 

In [29]:
cai.set_cai_index(codon_tables_w)

In [30]:
print(round(cai.cai_for_gene(GAL4),3))
print(round(cai.cai_for_gene(PPR1),3))
print(round(cai.cai_for_gene(GPD1),3))

0.701
0.718
0.812


In [31]:
print(round(CAI(GAL4, weights=codon_tables_w),3))
print(round(CAI(PPR1, weights=codon_tables_w),3)) 
print(round(CAI(GPD1, weights=codon_tables_w),3)) 

0.701
0.719
0.813


Results are rather close, but Genscript values are systematically higher.

| Gene 	| Genscript | CAI(codon_tables_w)|Biopython(codon_tables_w)|
|------	|-----------|--------------------|-------------------------|
| GAL4 	| 0.71      | 0.701              | 0.701                   | 
| PPR1 	| 0.73      | 0.719              | 0.718                   |
| GPD1 	| 0.82      | 0.813              | 0.812                   |

The difference is surprising. Either, the datasets used are not identical or the implementation is different.


Some other tools were tested as well:


### CAIcal SERVER
This is a fairly recent tool. "using the recent computer implementation proposed by Xia"
accepts codon usage table in "codon usage database format".

http://genomes.urv.es/CAIcal. If you set the reference data to the "kazusa" matrix above your get this result:

| Gene 	| Genscript | CAI(codon_tables_w)|Biopython(codon_tables_w)|CAIcal  |
|------	|-----------|--------------------|-------------------------|--------|
| GAL4 	| 0.71      | 0.701              | 0.701                   | 0.761  |
| PPR1 	| 0.73      | 0.719              | 0.718                   | 0.770  |
| GPD1 	| 0.82      | 0.813              | 0.812                   | 0.813  |

The lower value is higher than the ones produced by the other tools.

### COUSIN (COdon Usage Similarity INdex)
yet another tool is the the COUSIN server.
It accepts codon usage table in "kazusa" format
http://cousin.ird.fr/calculation.php

This tool produces two results CAI 18 and 59. The relevant measure here appears
to be CAI 59. This tool could have better documentation.

| Gene | Genscript | CAI(codon_tables_w)|Biopython(codon_tables_w)|CAIcal  | COUSIN |
|------|-----------|--------------------|-------------------------|--------| ------ |
| GAL4 | 0.71      | 0.701              | 0.701                   | 0.761  | 0.701  |
| PPR1 | 0.73      | 0.719              | 0.718                   | 0.770  | 0.718  |
| GPD1 | 0.82      | 0.813              | 0.812                   | 0.813  | 0.814  |

The COUSIN server gives a result close to the one of CAI and Biopython.

### Codon Adaptation Index Calculator (CAICalculator)
Another tool available online is the [Codon Adaptation Index Calculator](https://www.biologicscorp.com/tools/CAICalculator). 
This tool has a "codon usage table of Saccharomyces cerevisiae (Yeast)" option.


| Gene | Genscript | CAI(codon_tables_w)|Biopython(codon_tables_w)|CAIcal  | COUSIN | CAICalculator |
|------|-----------|--------------------|-------------------------|--------| ------ | ------------- |
| GAL4 | 0.71      | 0.701              | 0.701                   | 0.761  | 0.701  | 0.71          |
| PPR1 | 0.73      | 0.719              | 0.718                   | 0.770  | 0.718  | 0.73          |
| GPD1 | 0.82      | 0.813              | 0.812                   | 0.813  | 0.814  | 0.82          |


This tool produce the exact same result as Genscript does. This tool belongs to a company that I do not think have
any affiliations with Genscript. This means that both companies most likely use the same data and software.

This tool displays the reference data as opposed to the Genscript tool (see the cell below). 

Tha data used is in fact the same data as in the kazusa matrix above.

The differences between the python modules and Genscript/CAICalculator must be in the implementation.

In [32]:
t = """\
triplet aa fraction rscu weight freq number
TTT F 0.59 1.17 1.000 26.1 170666
TTG L 0.29 1.72 1.000 27.2 177573
TAT Y 0.56 1.12 1.000 18.8 122728
TAG * 0.23 0.68 0.479 0.5 3312
TGT C 0.63 1.26 1.000 8.1 52903
TGG W 1.00 1.00 1.000 10.4 67789
TCT S 0.26 1.58 1.000 23.5 153557
TCG S 0.10 0.58 0.364 8.6 55951
ATT I 0.46 1.39 1.000 30.1 196893
ATG M 1.00 1.00 1.000 20.9 136805
AAT N 0.59 1.18 1.000 35.7 233124
AAG K 0.42 0.85 0.736 30.8 201361
AGT S 0.16 0.95 0.602 14.2 92466
AGG R 0.21 1.25 0.433 9.2 60289
ACT T 0.34 1.38 1.000 20.3 132522
ACG T 0.14 0.54 0.393 8.0 52045
GTT V 0.39 1.56 1.000 22.1 144243
GTG V 0.19 0.76 0.488 10.8 70337
GAT D 0.65 1.30 1.000 37.6 245641
GAG E 0.30 0.59 0.422 19.2 125717
GGT G 0.47 1.89 1.000 23.9 156109
GGG G 0.12 0.48 0.252 6.0 39359
GCT A 0.38 1.51 1.000 21.2 138358
GCG A 0.11 0.44 0.292 6.2 40358
CTT L 0.13 0.77 0.451 12.3 80076
CTG L 0.11 0.66 0.386 10.5 68494
CAT H 0.64 1.27 1.000 13.6 89007
CAG Q 0.31 0.61 0.444 12.1 79121
CGT R 0.14 0.87 0.300 6.4 41791
CGG R 0.04 0.24 0.082 1.7 11351
CCT P 0.31 1.23 0.738 13.5 88263
CCG P 0.12 0.48 0.289 5.3 34597
TTA L 0.28 1.66 0.962 26.2 170884
TTC F 0.41 0.83 0.706 18.4 120510
TAA * 0.47 1.41 1.000 1.1 6913
TAC Y 0.44 0.88 0.787 14.8 96596
TGA * 0.30 0.91 0.643 0.7 4447
TGC C 0.37 0.74 0.588 4.8 31095
TCA S 0.21 1.26 0.795 18.7 122028
TCC S 0.16 0.96 0.605 14.2 92923
ATA I 0.27 0.82 0.590 17.8 116254
ATC I 0.26 0.79 0.570 17.2 112176
AAA K 0.58 1.15 1.000 41.9 273618
AAC N 0.41 0.82 0.696 24.8 162199
AGA R 0.48 2.89 1.000 21.3 139081
AGC S 0.11 0.66 0.415 9.8 63726
ACA T 0.30 1.21 0.876 17.8 116084
ACC T 0.22 0.87 0.628 12.7 83207
GTA V 0.21 0.84 0.533 11.8 76927
GTC V 0.21 0.84 0.533 11.8 76947
GAA E 0.70 1.41 1.000 45.6 297944
GAC D 0.35 0.70 0.538 20.2 132048
GGA G 0.21 0.86 0.456 10.9 71216
GGC G 0.19 0.77 0.409 9.8 63903
GCA A 0.29 1.16 0.765 16.2 105910
GCC A 0.22 0.90 0.595 12.6 82357
CTA L 0.14 0.85 0.493 13.4 87619
CTC L 0.06 0.34 0.200 5.4 35545
CAA Q 0.69 1.39 1.000 27.3 178251
CAC H 0.36 0.73 0.571 7.8 50785
CGA R 0.07 0.41 0.141 3.0 19562
CGC R 0.06 0.35 0.122 2.6 16993
CCA P 0.42 1.67 1.000 18.3 119641
CCC P 0.15 0.62 0.370 6.8 44309"""

The CAICalculator data was read into a dataframe and some dicts:

In [33]:
gbpln = pd.read_csv(StringIO(t.strip()), sep=" ")

In [34]:
CAICalculator_w = dict(zip(gbpln["triplet"],round(gbpln["weight"],3)))
CAICalculator_RSCU = dict(zip(gbpln["triplet"],round(gbpln["rscu"],3)))
CAICalculator_number = dict(zip(gbpln["triplet"],round(gbpln["number"],3)))

In [35]:
lst = []
for key in codon_tables_w:
    lst.append((key, round(codon_tables_w[key],3), round(CAICalculator_w[key],3) ))

In [36]:
pd.DataFrame(lst, columns=['Triplet', 'codon_tables_w', 'CAICalculator'])

,Triplet,codon_tables_w,CAICalculator
0,TAA,1.000,1.000
1,TAG,0.489,0.479
2,TGA,0.638,0.643
3,GCA,0.763,0.765
4,GCC,0.579,0.595
...,...,...,...
59,GTG,0.487,0.488
60,GTT,1.000,1.000
61,TGG,1.000,1.000
62,TAC,0.786,0.787


In [37]:
print(round(CAI(GAL4, weights=CAICalculator_w),3))
print(round(CAI(PPR1, weights=CAICalculator_w),3)) 
print(round(CAI(GPD1, weights=CAICalculator_w),3)) 

0.701
0.718
0.814


In [38]:
del CAICalculator_RSCU["TAG"]
del CAICalculator_RSCU["TAA"]
del CAICalculator_RSCU["TGA"]

In [39]:
print(round(CAI(GAL4[3:-3], RSCUs=CAICalculator_RSCU),3))
print(round(CAI(PPR1[3:-3], RSCUs=CAICalculator_RSCU),3)) 
print(round(CAI(GPD1[3:-3], RSCUs=CAICalculator_RSCU),3)) 

0.701
0.718
0.815


In [40]:
cai.set_cai_index(CAICalculator_w)

In [41]:
print(round(cai.cai_for_gene(GAL4),3))
print(round(cai.cai_for_gene(PPR1),3))
print(round(cai.cai_for_gene(GPD1),3))

0.701
0.717
0.813


The result for CAI and Biopython with the new reference data (CAICalculator_w) is again similar to the COUSIN results.

| Gene | Genscript | CAI(CAICalculator_w)|Biopython(CAICalculator_w)|CAIcal  | COUSIN | CAICalculator |
|------|-----------|---------------------|--------------------------|--------| ------ | ------------- |
| GAL4 | 0.71      | 0.701               | 0.701                    | 0.761  | 0.701  | 0.71          |
| PPR1 | 0.73      | 0.718               | 0.717                    | 0.770  | 0.718  | 0.73          |
| GPD1 | 0.82      | 0.814               | 0.813                    | 0.813  | 0.814  | 0.82          |

We can note a small difference between the CAICalculator_w and codon_tables_w. Perhaps this is due to rounding errors.

This could be tested by generating the reference data by using the counts of each codon.

In [42]:
gencode = {'*': ('TAA', 'TAG', 'TGA'),
 'A': ('GCA', 'GCC', 'GCG', 'GCT'),
 'C': ('TGC', 'TGT'),
 'D': ('GAC', 'GAT'),
 'E': ('GAA', 'GAG'),
 'F': ('TTC', 'TTT'),
 'G': ('GGA', 'GGC', 'GGG', 'GGT'),
 'H': ('CAC', 'CAT'),
 'I': ('ATA', 'ATC', 'ATT'),
 'K': ('AAA', 'AAG'),
 'L': ('CTA', 'CTC', 'CTG', 'CTT', 'TTA', 'TTG'),
 'M': ('ATG',),
 'N': ('AAC', 'AAT'),
 'P': ('CCA', 'CCC', 'CCG', 'CCT'),
 'Q': ('CAA', 'CAG'),
 'R': ('AGA', 'AGG', 'CGA', 'CGC', 'CGG', 'CGT'),
 'S': ('AGC', 'AGT', 'TCA', 'TCC', 'TCG', 'TCT'),
 'T': ('ACA', 'ACC', 'ACG', 'ACT'),
 'V': ('GTA', 'GTC', 'GTG', 'GTT'),
 'W': ('TGG',),
 'Y': ('TAC', 'TAT')}

In [43]:
kazusa_w_from_counts = {}
for aa, triplets in gencode.items():
    counts = [CAICalculator_number[cdn] for cdn in triplets]
    maxm = max(counts)
    for cdn in triplets:
        kazusa_w_from_counts[cdn] = CAICalculator_number[cdn]/maxm

In [44]:
print(round(CAI(GAL4, weights=kazusa_w_from_counts),3))
print(round(CAI(PPR1, weights=kazusa_w_from_counts),3)) 
print(round(CAI(GPD1, weights=kazusa_w_from_counts),3)) 

0.701
0.718
0.814


It seems that the new data did not make any difference.

In [45]:
CAICalculator_w

{'TTT': 1.0,
 'TTG': 1.0,
 'TAT': 1.0,
 'TAG': 0.479,
 'TGT': 1.0,
 'TGG': 1.0,
 'TCT': 1.0,
 'TCG': 0.364,
 'ATT': 1.0,
 'ATG': 1.0,
 'AAT': 1.0,
 'AAG': 0.736,
 'AGT': 0.602,
 'AGG': 0.433,
 'ACT': 1.0,
 'ACG': 0.393,
 'GTT': 1.0,
 'GTG': 0.488,
 'GAT': 1.0,
 'GAG': 0.422,
 'GGT': 1.0,
 'GGG': 0.252,
 'GCT': 1.0,
 'GCG': 0.292,
 'CTT': 0.451,
 'CTG': 0.386,
 'CAT': 1.0,
 'CAG': 0.444,
 'CGT': 0.3,
 'CGG': 0.082,
 'CCT': 0.738,
 'CCG': 0.289,
 'TTA': 0.962,
 'TTC': 0.706,
 'TAA': 1.0,
 'TAC': 0.787,
 'TGA': 0.643,
 'TGC': 0.588,
 'TCA': 0.795,
 'TCC': 0.605,
 'ATA': 0.59,
 'ATC': 0.57,
 'AAA': 1.0,
 'AAC': 0.696,
 'AGA': 1.0,
 'AGC': 0.415,
 'ACA': 0.876,
 'ACC': 0.628,
 'GTA': 0.533,
 'GTC': 0.533,
 'GAA': 1.0,
 'GAC': 0.538,
 'GGA': 0.456,
 'GGC': 0.409,
 'GCA': 0.765,
 'GCC': 0.595,
 'CTA': 0.493,
 'CTC': 0.2,
 'CAA': 1.0,
 'CAC': 0.571,
 'CGA': 0.141,
 'CGC': 0.122,
 'CCA': 1.0,
 'CCC': 0.37}